In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
from sklearn.metrics import confusion_matrix, mean_squared_error
import os
from Graphs import matrix_to_graph, graph_to_matrix, ErdosRenyiGraph, getLambda
from ShortestDistanceAlgorithms import shortestDistance_allNodes_networkx, shortestDistance_allNodes_Bourgain, shortestDistance_allNodes_Sarma
from Models import build

from itertools import chain

In [ ]:
def generateERSamples_inner(num_graphs,n,lbd):

    #largest_component_sizes = []
    #lbds = []

    samples_x = []
    samples_y_actual = []
    samples_y_Bourgain = []
    samples_y_Sarma1 = []
    samples_y_Sarma2 = []
    samples_y_Sarma3 = []
    #samples_y_Sarma4 = []
    samples_edge_index = []

    #max_lbd = getLambda(n)
    k = 0
    n_rejected1 = 0
    n_rejected2 = 0
    while k < num_graphs:
        try:
            object,directed,weighted = ErdosRenyiGraph(n,lbd/n)
            components = list(nx.strongly_connected_components(object))
            largest_component = max(components, key=len)
            n_nodes = len(largest_component)
            r = int(np.floor(np.sqrt(n)))
            if n_nodes >= max(r,10):
                #largest_component_sizes.append(n_nodes)
                #lbds.append(lbd)
                object = object.subgraph(largest_component)
                object = nx.relabel_nodes(object, {node: index for index, node in enumerate(object.nodes())})
                matrix = graph_to_matrix(object)
                seeds = np.random.choice(range(n_nodes),size=r,replace=False)
                x = np.zeros((n_nodes,r))
                y_actual = np.zeros((n_nodes,r))
                y_Bourgain = np.zeros((n_nodes,r))
                y_Sarma1 = np.zeros((n_nodes,r))
                y_Sarma2 = np.zeros((n_nodes,r))
                y_Sarma3 = np.zeros((n_nodes,r))
                #y_Sarma4 = np.zeros((n_nodes,r))
                M = n_nodes
                for i in range(r):
                    u = seeds[i]
                    #support = [n for n in range(matrix.shape[0]) if np.count_nonzero(matrix[n]) >= 2 and n != u] # u is removed from the support for sampling seed sets
                    #print(support)
                    x[u,i] = 1
                    y_actual[:,i] = shortestDistance_allNodes_networkx(object,u)
                    if n <= 200:
                        y_Bourgain[:,i] = shortestDistance_allNodes_Bourgain(matrix,u)
                        y_Sarma1[:,i] = shortestDistance_allNodes_Sarma(matrix,u,1)
                        y_Sarma2[:,i] = shortestDistance_allNodes_Sarma(matrix,u,2)
                        y_Sarma3[:,i] = shortestDistance_allNodes_Sarma(matrix,u,3)
                    #y_Sarma4[:,i] = shortestDistance_allNodes_Bourgain(matrix,u,4)
                    #y_Sarma4 = np.where(y_Sarma4 == float('inf'), M, y_Sarma4)
                if n <= 200:
                    y_Bourgain = np.where(y_Bourgain == float('inf'), M, y_Bourgain)
                    y_Sarma1 = np.where(y_Sarma1 == float('inf'), M, y_Sarma1)
                    y_Sarma2 = np.where(y_Sarma2 == float('inf'), M, y_Sarma2)
                    y_Sarma3 = np.where(y_Sarma3 == float('inf'), M, y_Sarma3)
                samples_x.append(torch.tensor(x.astype(np.float32), requires_grad=True))
                samples_y_actual.append(torch.tensor(y_actual).to(torch.float32))
                samples_y_Bourgain.append(y_Bourgain)
                samples_y_Sarma1.append(y_Sarma1)
                samples_y_Sarma2.append(y_Sarma2)
                samples_y_Sarma3.append(y_Sarma3)
                samples_edge_index.append(torch.tensor(np.array(list(object.edges())).T).to(torch.int64))
                k += 1
            else:
                n_rejected2 += 1
        except:
            n_rejected1 += 1
    
    print('Number of graphs rejected because Bourgain\'s and Sarma\'s algorithms yield errors: ',n_rejected1)
    print('Number of graphs rejected because the largest component has insufficient size: ',n_rejected2)

    return samples_x, [samples_y_actual, samples_y_Bourgain, samples_y_Sarma1, samples_y_Sarma2, samples_y_Sarma3], samples_edge_index, None ## None for sample weights

def generateERSamples(n_train,n_val,n_test,n,lbd):
    print('Generating training data...')
    train = generateERSamples_inner(n_train,n,lbd)
    print('Generating validation data...')
    val = generateERSamples_inner(n_val,n,lbd)
    print('Generating test data...')
    test = generateERSamples_inner(n_test,n,lbd)
    return [train, val, test]

In [ ]:
def predict(gpu_bool,model,criterion_type,samples_x,samples_edge_index=None,samples_weights=None):
    y_pred = []
    if gpu_bool:
        model = model.to('cuda:1')
    model.eval()
    with torch.no_grad():
        if model.out_channels == 1:
            if model.name == 'mlp':
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1))  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
            elif samples_weights == None:
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                        edge_index = samples_edge_index[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                        edge_index = samples_edge_index[i]
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1),edge_index)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
            else:
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                        edge_index = samples_edge_index[i].to('cuda:1')
                        weights = samples_weights[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                        edge_index = samples_edge_index[i]
                        weights = samples_weights[i]
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1),edge_index,weights)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
        else:
            if model.name == 'mlp':
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                    out = model(x)  # Perform a single forward pass.
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
            elif samples_weights == None:
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                        edge_index = samples_edge_index[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                        edge_index = samples_edge_index[i]
                    out = model(x,edge_index)  # Perform a single forward pass.
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
            else:
                for i in range(len(samples_x)):
                    if gpu_bool:
                        x = samples_x[i].to('cuda:1')
                        edge_index = samples_edge_index[i].to('cuda:1')
                        weights = samples_weights[i].to('cuda:1')
                    else:
                        x = samples_x[i]
                        edge_index = samples_edge_index[i]
                        weights = samples_weights[i]
                    out = model(x,edge_index,weights)  # Perform a single forward pass.
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
    model = model.to('cpu')
    return y_pred

def predict_allBatches(model,criterion_type,samples):
    gpu_bool = torch.cuda.is_available()
    y_pred_train = predict(gpu_bool, model, criterion_type, samples[0][0], samples[0][2], samples[0][3])
    y_pred_val = predict(gpu_bool, model, criterion_type, samples[1][0], samples[1][2], samples[1][3])
    y_pred_test = predict(gpu_bool, model, criterion_type, samples[2][0], samples[2][2], samples[2][3])
    return y_pred_train,y_pred_val,y_pred_test

In [ ]:
def test(gpu_bool,model,criterion,criterion_type,samples_x,samples_y,samples_edge_index = None,samples_weights = None):
    t_loss = 0
    total_samples = 0
    y_pred = []
    model.eval()
    with torch.no_grad():
        if model.out_channels == 1:
            if samples_edge_index == None:
                for x,y in list(zip(samples_x,samples_y)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1))  # Perform a single forward pass.
                        t_loss += criterion[0](out.squeeze(), y[:,j])
                        total_samples += 1
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1','mse-mse']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
            elif samples_weights == None:
                for x,y,edge_index in list(zip(samples_x,samples_y,samples_edge_index)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                        edge_index = edge_index.to('cuda:1')
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1),edge_index)  # Perform a single forward pass.
                        t_loss += criterion[0](out.squeeze(), y[:,j])
                        total_samples += 1
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1','mse-mse']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
            else:
                for x,y,edge_index,weights in list(zip(samples_x,samples_y,samples_edge_index,samples_weights)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                        edge_index = edge_index.to('cuda:1')
                        weights = weights.to('cuda:1')
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1),edge_index,weights)  # Perform a single forward pass.
                        t_loss += criterion[0](out.squeeze(), y[:,j])
                        total_samples += 1
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1','mse-mse']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
        else:
            if samples_edge_index == None:
                for x,y in list(zip(samples_x,samples_y)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                    out = model(x)  # Perform a single forward pass.
                    if criterion_type in ['bce']:
                        t_loss += criterion[0](out, torch.stack((1-y, y)).T)
                    elif criterion_type == 'mse-mse':
                        t_loss += 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                    elif criterion_type in ['ce','multimargin']:
                        t_loss += criterion[0](out, y) ## classification
                    else:
                        t_loss += criterion[0](out.squeeze(), y)
                    total_samples += 1
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1','mse-mse']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
            elif samples_weights == None:
                for x,y,edge_index in list(zip(samples_x,samples_y,samples_edge_index)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                        edge_index = edge_index.to('cuda:1')
                    out = model(x,edge_index)  # Perform a single forward pass.
                    if criterion_type in ['bce']:
                        t_loss += criterion[0](out, torch.stack((1-y, y)).T)
                    elif criterion_type == 'mse-mse':
                        t_loss += 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                    elif criterion_type in ['ce','multimargin']:
                        t_loss += criterion[0](out, y) ## classification
                    else:
                        t_loss += criterion[0](out.squeeze(), y)
                    total_samples += 1
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1','mse-mse']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
            else:
                for x,y,edge_index,weights in list(zip(samples_x,samples_y,samples_edge_index,samples_weights)):
                    if gpu_bool:
                        x = x.to('cuda:1')
                        y = y.to('cuda:1')
                        edge_index = edge_index.to('cuda:1')
                        weights = weights.to('cuda:1')
                    out = model(x,edge_index,weights)  # Perform a single forward pass.
                    if criterion_type in ['bce']:
                        t_loss += criterion[0](out, torch.stack((1-y, y)).T)
                    elif criterion_type == 'mse-mse':
                        t_loss += 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                    elif criterion_type in ['ce','multimargin']:
                        t_loss += criterion[0](out, y) ## classification
                    else:
                        t_loss += criterion[0](out.squeeze(), y)
                    total_samples += 1
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1','mse-mse']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())

    t_loss = t_loss.cpu()/total_samples
    if criterion_type in ['mse','l2','l1','mse-mse']:
        return t_loss, None, None, None
    else:
        y_true = np.array([y.cpu() for y in samples_y])
        y_pred = np.array(y_pred)
        t_accuracy = sum(y_true == y_pred)/len(y_true)
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        t_sensitivity = tp / (tp + fn)
        t_specificity = tn / (tn + fp)
        return t_loss, t_accuracy, t_sensitivity, t_specificity

def train(gpu_bool,model,criterion,optimizer,scheduler,criterion_type,scheduler_type,x_train,x_val,y_train,y_val,edge_index_train=None,edge_index_val=None,weights_train=None,weights_val=None):
    model.train()
    if model.out_channels == 1:
        if edge_index_train == None:
            for x,y in list(zip(x_train,y_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                for j in range(x.shape[1]):
                    out = model(x[:,j].reshape(len(x[:,j]),1))  # Perform a single forward pass.
                    t_loss = criterion[0](out.squeeze(), y[:,j])
                    t_loss.backward()  # Derive gradients
                    optimizer.step()  # Update parameters based on gradients.
        elif weights_train == None:
            for x,y,edge_index in list(zip(x_train,y_train,edge_index_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                    edge_index = edge_index.to('cuda:1')
                for j in range(x.shape[1]):
                    out = model(x[:,j].reshape(len(x[:,j]),1),edge_index)  # Perform a single forward pass
                    t_loss = criterion[0](out.squeeze(), y[:,j])
                    t_loss.backward()  # Derive gradients
                    optimizer.step()  # Update parameters based on gradients.
        else:
            for x,y,edge_index,weights in list(zip(x_train,y_train,edge_index_train,weights_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                    edge_index = edge_index.to('cuda:1')
                    weights = weights.to('cuda:1')
                for j in range(x.shape[1]):
                    out = model(x[:,j].reshape(len(x[:,j]),1),edge_index,weights)  # Perform a single forward pass.
                    t_loss = criterion[0](out.squeeze(), y[:,j])
                    t_loss.backward()  # Derive gradients
                    optimizer.step()  # Update parameters based on gradients.
    else:
        if edge_index_train == None:
            for x,y in list(zip(x_train,y_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                out = model(x)  # Perform a single forward pass.
                if criterion_type in ['bce']:
                    t_loss = criterion[0](out, torch.stack((1-y, y)).T)
                elif criterion_type == 'mse-mse':
                    t_loss = 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                elif criterion_type in ['ce','multimargin']:
                    t_loss = criterion[0](out, y) ## classification
                else:
                    t_loss = criterion[0](out.squeeze(), y)
                t_loss.backward()  # Derive gradients
                optimizer.step()  # Update parameters based on gradients.
        elif weights_train == None:
            for x,y,edge_index in list(zip(x_train,y_train,edge_index_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                    edge_index = edge_index.to('cuda:1')
                out = model(x,edge_index)  # Perform a single forward pass
                if criterion_type in ['bce']:
                    t_loss = criterion[0](out, torch.stack((1-y, y)).T)
                elif criterion_type == 'mse-mse':
                    t_loss = 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                elif criterion_type in ['ce','multimargin']:
                    t_loss = criterion[0](out, y) ## classification
                else:
                    t_loss = criterion[0](out.squeeze(), y)
                t_loss.backward()  # Derive gradients
                optimizer.step()  # Update parameters based on gradients.
        else:
            for x,y,edge_index,weights in list(zip(x_train,y_train,edge_index_train,weights_train)):
                optimizer.zero_grad()  # Clear gradients.
                if gpu_bool:
                    x = x.to('cuda:1')
                    y = y.to('cuda:1')
                    edge_index = edge_index.to('cuda:1')
                    weights = weights.to('cuda:1')
                out = model(x,edge_index,weights)  # Perform a single forward pass.
                if criterion_type in ['bce']:
                    t_loss = criterion[0](out, torch.stack((1-y, y)).T)
                elif criterion_type == 'mse-mse':
                    t_loss = 100*criterion[0](out[:, ::2], y[:, ::2]) + criterion[1](out[:, 1::2], y[:, 1::2])
                elif criterion_type in ['ce','multimargin']:
                    t_loss = criterion[0](out, y) ## classification
                else:
                    t_loss = criterion[0](out.squeeze(), y)
                t_loss.backward()  # Derive gradients
                optimizer.step()  # Update parameters based on gradients.

    train_loss, train_accuracy, train_sensitivity, train_specificity = test(gpu_bool,model,criterion,criterion_type,x_train,y_train,edge_index_train,weights_train)
    v_loss, v_accuracy, v_sensitivity, v_specificity = test(gpu_bool,model,criterion,criterion_type,x_val,y_val,edge_index_val,weights_val)

    if scheduler_type in ['step','exponential','cyclic','cosine']:
        scheduler[0].step()
    elif scheduler_type == 'reduce_on_plateau': 
        scheduler[0].step(v_loss)
    elif scheduler_type == 'cyclic-cosine':
        scheduler[0].step()
        scheduler[1].step()
    return train_loss,train_accuracy,train_sensitivity,train_specificity,v_loss,v_accuracy,v_sensitivity,v_specificity

In [ ]:
def evaluate_inner(n,data_name,y,y_pred1,y_pred2):

    y_pred1 = np.array(list(chain(*[np.ravel(arr) for arr in y_pred1])))
    y_pred2 = np.array(list(chain(*[np.ravel(arr) for arr in y_pred2])))
    y_actual = np.array(list(chain(*[np.ravel(arr) for arr in y[0]])))
    diff_pred1 = y_actual - y_pred1
    diff_pred2 = y_actual - y_pred2

    values, base = np.histogram(diff_pred1, bins=100)
    cumulative = np.cumsum(values)
    plt.plot(base[:-1], cumulative, label='GNN, out_channels = floor(sqrt(n))', alpha = 0.75)
    
    values, base = np.histogram(diff_pred2, bins=100)
    cumulative = np.cumsum(values)
    plt.plot(base[:-1], cumulative, label='GNN, out_channels = 1', alpha = 0.75)

    if n <= 200:

        y_Bourgain = np.array(list(chain(*[np.ravel(arr) for arr in y[1]])))
        y_Sarma1 = np.array(list(chain(*[np.ravel(arr) for arr in y[2]])))
        y_Sarma2 = np.array(list(chain(*[np.ravel(arr) for arr in y[3]])))
        y_Sarma3 = np.array(list(chain(*[np.ravel(arr) for arr in y[4]])))
        diff_Bourgain = y_actual-y_Bourgain
        diff_Sarma1 = y_actual-y_Sarma1
        diff_Sarma2 = y_actual-y_Sarma2
        diff_Sarma3 = y_actual-y_Sarma3

        values, base = np.histogram(diff_Bourgain, bins=100)
        cumulative = np.cumsum(values)
        plt.plot(base[:-1], cumulative, label='Bourgain', alpha = 0.75)

        values, base = np.histogram(diff_Sarma1, bins=100)
        cumulative = np.cumsum(values)
        plt.plot(base[:-1], cumulative, label='Sarma, k = 1', alpha = 0.75)

        values, base = np.histogram(diff_Sarma2, bins=100)
        cumulative = np.cumsum(values)
        plt.plot(base[:-1], cumulative, label='Sarma, k = 2', alpha = 0.75)

        values, base = np.histogram(diff_Sarma3, bins=100)
        cumulative = np.cumsum(values)
        plt.plot(base[:-1], cumulative, label='Sarma, k = 3', alpha = 0.75)

    plt.xlabel("Actual Distance - Predicted Distance")
    plt.ylabel("Cummulative Frequency")
    plt.title(data_name+", n = "+str(n)+", lambda = "+str(lbd))
    plt.legend()
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

    plt.scatter(y_actual, y_pred1, label='GNN, out_channels = floor(sqrt(n))', alpha = 0.1)
    plt.scatter(y_actual, y_pred2, label='GNN, out_channels = 1', alpha = 0.1)
    plt.xlabel("Actual Distance")
    plt.ylabel('Predicted Distance')
    plt.title(data_name+", n = "+str(n)+", lambda = "+str(lbd))
    plt.legend()
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

    plt.hist(y_actual, edgecolor='white', alpha=0.4)
    #sns.kdeplot(y_actual, fill=True, alpha=0.2)
    plt.xlabel("Actual Distance")
    plt.ylabel('Density')
    plt.title(data_name+", n = "+str(n)+", lambda = "+str(lbd))
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

    return diff_pred1
    
def evaluate(n,model1,model2,criterion_type,samples):
    y_pred_train1,y_pred_val1,y_pred_test1 = predict_allBatches(model1,criterion_type,samples)
    y_pred_train2,y_pred_val2,y_pred_test2 = predict_allBatches(model2,criterion_type,samples) 
    diff_pred_train = evaluate_inner(n,'Training Data',samples[0][1],y_pred_train1,y_pred_train2)
    diff_pred_val = evaluate_inner(n,'Validation Data',samples[1][1],y_pred_val1,y_pred_val2)
    diff_pred_test = evaluate_inner(n,'Test Data',samples[2][1],y_pred_test1,y_pred_test2)

    sns.kdeplot(diff_pred_train, fill=True, label="Training Data", alpha=0.2)
    sns.kdeplot(diff_pred_val, fill=True, label="Validation Data", alpha=0.2)
    sns.kdeplot(diff_pred_test, fill=True, label="Test Data", alpha=0.2)

    plt.xlabel("Actual Distance - Predicted Distance")
    plt.ylabel('Density')
    plt.title("n = "+str(n)+", lambda = "+str(lbd))
    plt.legend()
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

In [ ]:
def run(samples,model,criterion_type,optimizer_type,scheduler_type,num_epochs=100,early_stopping_patience=None,save_model = False):

    title = 'out_channels = floor(sqrt(n))'
    gpu_bool = torch.cuda.is_available()

    x_train = samples[0][0]
    x_val = samples[1][0]
    x_test = samples[2][0]
    y_train = samples[0][1][0]
    y_val = samples[1][1][0]
    y_test = samples[2][1][0]
    edge_index_train = samples[0][2]
    edge_index_val = samples[1][2]
    edge_index_test = samples[2][2]
    weights_train = samples[0][3]
    weights_val = samples[1][3]
    weights_test = samples[2][3]

    in_channels = x_train[0].shape[1]
    out_channels = y_train[0].shape[1]
    if isinstance(model, str):
        model_type = model
        model,criterion,optimizer,scheduler = build(in_channels, out_channels, model_type,criterion_type,optimizer_type,scheduler_type)
    else:
        model_type = model.name
        _,criterion,optimizer,scheduler = build(in_channels, out_channels, model, criterion_type,optimizer_type,scheduler_type)
    print(model)

    if gpu_bool:
        model = model.to('cuda:1')

    train_loss = []
    train_acc = []
    train_sen = []
    train_spec = []
    val_loss = []
    val_acc = []
    val_sen = []
    val_spec = []

    if early_stopping_patience != None:
        best_val_loss = float('inf')
        best_epoch = 0
        no_improvement_count = 0

    for epoch in range(1, num_epochs+1):
        if model_type == 'mlp':
            t_loss,t_acc,t_sen,t_spec, v_loss, v_acc, v_sen, v_spec = train(gpu_bool,model,criterion,optimizer,scheduler,criterion_type,scheduler_type,x_train,x_val,y_train,y_val)
        else:
            t_loss,t_acc,t_sen,t_spec, v_loss, v_acc, v_sen, v_spec = train(gpu_bool,model,criterion,optimizer,scheduler,criterion_type,optimizer_type,x_train,x_val,y_train,y_val,edge_index_train,edge_index_val,weights_train,weights_val)
        train_loss.append(t_loss)
        train_acc.append(t_acc)
        train_sen.append(t_sen)
        train_spec.append(t_spec)
        val_loss.append(v_loss)
        val_acc.append(v_acc)
        val_sen.append(v_sen)
        val_spec.append(v_spec)
        if epoch % 10 == 0:
            if criterion_type in ['mse','l2','mse-mse']:
                print(f'Epoch: {epoch:03d}, Training Loss (MSE): {t_loss:.4f}, Validation Loss (MSE): {v_loss:.4f}')
            elif criterion_type == 'l1':
                print(f'Epoch: {epoch:03d}, Training Loss (MAE): {t_loss:.4f}, Validation Loss (MAE): {v_loss:.4f}')
            else:
                print(f'Epoch: {epoch:03d}, Training Loss: {t_loss:.4f}, Training Accuracy: {t_acc:.4f}, Training Sensitivity: {t_sen:.4f}, Training Specificity: {t_spec:.4f}, Validation Loss: {v_loss:.4f}, Validation Accuracy: {v_acc:.4f}, Validation Sensitivity: {v_sen:.4f}, Validation Specificity: {v_spec:.4f}')

        if early_stopping_patience != None:
            if v_loss < best_val_loss:
                best_val_loss = v_loss
                best_epoch = epoch
                no_improvement_count = 0
                torch.save(model.state_dict(), 'best_model_sqrt1.pth')
            else:
                no_improvement_count += 1
            if no_improvement_count >= early_stopping_patience:
                model.load_state_dict(torch.load('best_model_sqrt1.pth'))
                if gpu_bool:
                    model = model.to('cuda:1')
                break

    if model_type == 'mlp':
        test_loss, test_acc, test_sen, test_spec = test(gpu_bool,model,criterion,criterion_type,x_test, y_test)
    else:
        test_loss, test_acc, test_sen, test_spec = test(gpu_bool,model,criterion,criterion_type,x_test, y_test, edge_index_test, weights_test)
    
    if early_stopping_patience == None:
        if criterion_type in ['mse','l2','mse-mse']:
            print(f'Test Loss (MSE): {test_loss:03f}')
        elif criterion_type == 'l1':
            print(f'Test Loss (MAE): {test_loss:03f}')
        else:
            print(f'Test Loss: {test_loss:03f}, Test Accuracy: {test_acc:03f}, Test Sensitivity: {test_sen:03f}, Test Specificity: {test_spec:03f}')
    else:
        if criterion_type in ['mse','l2','mse-mse']:
            print(f'Best Epoch: {best_epoch:03d}, Test Loss (MSE): {test_loss:03f}')
        elif criterion_type == 'l1':
            print(f'Best Epoch: {best_epoch:03d}, Test Loss (MAE): {test_loss:03f}')
        else:
            print(f'Best Epoch: {best_epoch:03d}, Test Loss: {test_loss:03f}, Test Accuracy: {test_acc:03f}, Test Sensitivity: {test_sen:03f}, Test Specificity: {test_spec:03f}')

    x = range(1, len(train_loss)+1)
    plt.plot(x, train_loss, color = '#1f77b4', label = 'Training Loss', alpha = 0.75)
    plt.plot(x, val_loss, color = '#ff7f0e', label = 'Validation Loss', alpha = 0.75)
    if criterion_type in ['ce','bce','bcelogits','multimargin']:
        plt.plot(x, train_acc, color = '#2ca02c', label = 'Training Accuracy', alpha = 0.75)
        plt.plot(x, val_acc, color = '#d62728', label = 'Validation Accuracy', alpha = 0.75)
        plt.plot(x, train_sen, color = '#9467bd', label = 'Training Sensitivity', alpha = 0.75)
        plt.plot(x, val_sen, color = '#8c564b', label = 'Validation Sensitivity', alpha = 0.75)
        plt.plot(x, train_spec, color = '#e377c2', label = 'Training Specificity', alpha = 0.75)
        plt.plot(x, val_spec, color = '#7f7f7f', label = 'Validation Specificity', alpha = 0.75)
    plt.xlabel("Epoch")
    plt.ylabel("")
    plt.title("Training Results: "+title)
    plt.legend()
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

    model = model.to('cpu')
    if save_model:
        torch.save(model.state_dict(), 'trained_model_'+str(model.in_channels)+'_'+str(model.first_hidden_channels)+'_'+str(model.out_channels)+'_'+str(model.n_hidden_layers)+'.pth')
    
    return model
    #evaluate(title,model,criterion_type,samples)

In [ ]:
def run_out1(samples,model,criterion_type,optimizer_type,scheduler_type,num_epochs=100,early_stopping_patience=None,save_model = False):

    title = 'out_channels = 1'
    gpu_bool = torch.cuda.is_available()

    x_train = samples[0][0]
    x_val = samples[1][0]
    x_test = samples[2][0]
    y_train = samples[0][1][0]
    y_val = samples[1][1][0]
    y_test = samples[2][1][0]
    edge_index_train = samples[0][2]
    edge_index_val = samples[1][2]
    edge_index_test = samples[2][2]
    weights_train = samples[0][3]
    weights_val = samples[1][3]
    weights_test = samples[2][3]

    in_channels = 1
    out_channels = 1
    if isinstance(model, str):
        model_type = model
        model,criterion,optimizer,scheduler = build(in_channels, out_channels, model_type,criterion_type,optimizer_type,scheduler_type)
    else:
        model_type = model.name
        _,criterion,optimizer,scheduler = build(in_channels, out_channels, model, criterion_type,optimizer_type,scheduler_type)
    print(model)

    if gpu_bool:
        model = model.to('cuda:1')

    train_loss = []
    train_acc = []
    train_sen = []
    train_spec = []
    val_loss = []
    val_acc = []
    val_sen = []
    val_spec = []

    if early_stopping_patience != None:
        best_val_loss = float('inf')
        best_epoch = 0
        no_improvement_count = 0

    for epoch in range(1, num_epochs+1):
        if model_type == 'mlp':
            t_loss,t_acc,t_sen,t_spec, v_loss, v_acc, v_sen, v_spec = train(gpu_bool,model,criterion,optimizer,scheduler,criterion_type,scheduler_type,x_train,x_val,y_train,y_val)
        else:
            t_loss,t_acc,t_sen,t_spec, v_loss, v_acc, v_sen, v_spec = train(gpu_bool,model,criterion,optimizer,scheduler,criterion_type,optimizer_type,x_train,x_val,y_train,y_val,edge_index_train,edge_index_val,weights_train,weights_val)
        train_loss.append(t_loss)
        train_acc.append(t_acc)
        train_sen.append(t_sen)
        train_spec.append(t_spec)
        val_loss.append(v_loss)
        val_acc.append(v_acc)
        val_sen.append(v_sen)
        val_spec.append(v_spec)
        if epoch % 10 == 0:
            if criterion_type in ['mse','l2','mse-mse']:
                print(f'Epoch: {epoch:03d}, Training Loss (MSE): {t_loss:.4f}, Validation Loss (MSE): {v_loss:.4f}')
            elif criterion_type == 'l1':
                print(f'Epoch: {epoch:03d}, Training Loss (MAE): {t_loss:.4f}, Validation Loss (MAE): {v_loss:.4f}')
            else:
                print(f'Epoch: {epoch:03d}, Training Loss: {t_loss:.4f}, Training Accuracy: {t_acc:.4f}, Training Sensitivity: {t_sen:.4f}, Training Specificity: {t_spec:.4f}, Validation Loss: {v_loss:.4f}, Validation Accuracy: {v_acc:.4f}, Validation Sensitivity: {v_sen:.4f}, Validation Specificity: {v_spec:.4f}')

        if early_stopping_patience != None:
            if v_loss < best_val_loss:
                best_val_loss = v_loss
                best_epoch = epoch
                no_improvement_count = 0
                torch.save(model.state_dict(), 'best_model_sqrt1.pth')
            else:
                no_improvement_count += 1
            if no_improvement_count >= early_stopping_patience:
                model.load_state_dict(torch.load('best_model_sqrt1.pth'))
                if gpu_bool:
                    model = model.to('cuda:1')
                break

    if model_type == 'mlp':
        test_loss, test_acc, test_sen, test_spec = test(gpu_bool,model,criterion,criterion_type,x_test, y_test)
    else:
        test_loss, test_acc, test_sen, test_spec = test(gpu_bool,model,criterion,criterion_type,x_test, y_test, edge_index_test, weights_test)
    
    if early_stopping_patience == None:
        if criterion_type in ['mse','l2','mse-mse']:
            print(f'Test Loss (MSE): {test_loss:03f}')
        elif criterion_type == 'l1':
            print(f'Test Loss (MAE): {test_loss:03f}')
        else:
            print(f'Test Loss: {test_loss:03f}, Test Accuracy: {test_acc:03f}, Test Sensitivity: {test_sen:03f}, Test Specificity: {test_spec:03f}')
    else:
        if criterion_type in ['mse','l2','mse-mse']:
            print(f'Best Epoch: {best_epoch:03d}, Test Loss (MSE): {test_loss:03f}')
        elif criterion_type == 'l1':
            print(f'Best Epoch: {best_epoch:03d}, Test Loss (MAE): {test_loss:03f}')
        else:
            print(f'Best Epoch: {best_epoch:03d}, Test Loss: {test_loss:03f}, Test Accuracy: {test_acc:03f}, Test Sensitivity: {test_sen:03f}, Test Specificity: {test_spec:03f}')

    x = range(1, len(train_loss)+1)
    plt.plot(x, train_loss, color = '#1f77b4', label = 'Training Loss', alpha = 0.75)
    plt.plot(x, val_loss, color = '#ff7f0e', label = 'Validation Loss', alpha = 0.75)
    if criterion_type in ['ce','bce','bcelogits','multimargin']:
        plt.plot(x, train_acc, color = '#2ca02c', label = 'Training Accuracy', alpha = 0.75)
        plt.plot(x, val_acc, color = '#d62728', label = 'Validation Accuracy', alpha = 0.75)
        plt.plot(x, train_sen, color = '#9467bd', label = 'Training Sensitivity', alpha = 0.75)
        plt.plot(x, val_sen, color = '#8c564b', label = 'Validation Sensitivity', alpha = 0.75)
        plt.plot(x, train_spec, color = '#e377c2', label = 'Training Specificity', alpha = 0.75)
        plt.plot(x, val_spec, color = '#7f7f7f', label = 'Validation Specificity', alpha = 0.75)
    plt.xlabel("Epoch")
    plt.ylabel("")
    plt.title("Training Results: "+title)
    plt.legend()
    filenames = os.listdir(dir)
    if '0.png' not in filenames:
        plt.savefig(dir+'/0.png')
    else:
        filenames = sorted([int(f[:-4]) for f in filenames])
        plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
    plt.show()

    model = model.to('cpu')
    if save_model:
        torch.save(model.state_dict(), 'trained_model_'+str(model.in_channels)+'_'+str(model.first_hidden_channels)+'_'+str(model.out_channels)+'_'+str(model.n_hidden_layers)+'.pth')
    
    return model
    #evaluate(title,model,criterion_type,samples)

In [ ]:
def generate_one_ER_all_distances(n,lbd,print_summary = False):

    samples_x = []
    samples_y_actual = []
    samples_y_Bourgain = []
    samples_y_Sarma1 = []
    samples_y_Sarma2 = []
    samples_y_Sarma3 = []
    samples_edge_index = []

    n_rejected1 = 0
    n_rejected2 = 0
    while True:
        try:
            object,directed,weighted = ErdosRenyiGraph(n,lbd/n)
            components = list(nx.strongly_connected_components(object))
            largest_component = max(components, key=len)
            n_nodes = len(largest_component)
            if n_nodes >= 10:
                object = object.subgraph(largest_component)
                object = nx.relabel_nodes(object, {node: index for index, node in enumerate(object.nodes())})
                matrix = graph_to_matrix(object)
                y_actual_all = np.zeros((n_nodes,n_nodes))
                y_Bourgain_all = np.zeros((n_nodes,n_nodes))
                y_Sarma1_all = np.zeros((n_nodes,n_nodes))
                y_Sarma2_all = np.zeros((n_nodes,n_nodes))
                y_Sarma3_all = np.zeros((n_nodes,n_nodes))
                M = n_nodes
                for i in range(n_nodes):
                    y_actual_all[:,i] = shortestDistance_allNodes_networkx(object,i)
                    if n <= 200:
                        y_Bourgain_all[:,i] = shortestDistance_allNodes_Bourgain(matrix,i)
                        y_Sarma1_all[:,i] = shortestDistance_allNodes_Sarma(matrix,i,1)
                        y_Sarma2_all[:,i] = shortestDistance_allNodes_Sarma(matrix,i,2)
                        y_Sarma3_all[:,i] = shortestDistance_allNodes_Sarma(matrix,i,3)
                samples_x.append(torch.tensor(np.eye(n_nodes).astype(np.float32), requires_grad=True))
                samples_y_actual.append(torch.tensor(y_actual_all).to(torch.float32))
                samples_edge_index.append(torch.tensor(np.array(list(object.edges())).T).to(torch.int64))
                if n <= 200:
                    y_Bourgain_all = np.where(y_Bourgain_all == float('inf'), M, y_Bourgain_all)
                    y_Sarma1_all = np.where(y_Sarma1_all == float('inf'), M, y_Sarma1_all)
                    y_Sarma2_all = np.where(y_Sarma2_all == float('inf'), M, y_Sarma2_all)
                    y_Sarma3_all = np.where(y_Sarma3_all == float('inf'), M, y_Sarma3_all)
                    samples_y_Bourgain.append(y_Bourgain_all)
                    samples_y_Sarma1.append(y_Sarma1_all)
                    samples_y_Sarma2.append(y_Sarma2_all)
                    samples_y_Sarma3.append(y_Sarma3_all)
                break
            else:
                n_rejected2 += 1
        except:
            n_rejected1 += 1
    
    if print_summary:
        print('Number of graphs rejected because Bourgain\'s and Sarma\'s algorithms yield errors: ',n_rejected1)
        print('Number of graphs rejected because the largest component has insufficient size: ',n_rejected2)

    return samples_x, [samples_y_actual, samples_y_Bourgain, samples_y_Sarma1, samples_y_Sarma2, samples_y_Sarma3], samples_edge_index, None

def evaluate_one_graph_all_distances(n,model,criterion_type,samples,display_results = False):
    gpu_bool = torch.cuda.is_available()
    if model.out_channels == 1:
        y_pred = predict(gpu_bool, model, criterion_type, samples[0], samples[2], samples[3])
        samples_y_actual = samples[1][0]
        samples_y_Bourgain = samples[1][1]
        samples_y_Sarma1 = samples[1][2]
        samples_y_Sarma2 = samples[1][3]
        samples_y_Sarma3 = samples[1][4]
        title = 'GNN, out_channels = 1'
    else:
        samples_x = []
        samples_y_actual = []
        samples_y_Bourgain = []
        samples_y_Sarma1 = []
        samples_y_Sarma2 = []
        samples_y_Sarma3 = []
        samples_edge_index = []
        y_actual_all = samples[1][0][0]
        y_Bourgain_all = samples[1][1][0]
        y_Sarma1_all = samples[1][2][0]
        y_Sarma2_all = samples[1][3][0]
        y_Sarma3_all = samples[1][4][0]
        edge_index = samples[2][0]
        n_nodes = y_actual_all.shape[0]
        r = model.out_channels
        if n_nodes % r != 0:
            n_extra = int(np.ceil(n_nodes/r))*r - n_nodes
            extra_seeds = np.random.choice(range(n_nodes),size=n_extra,replace=True)
            shuffled_nodes = np.random.permutation(list(range(n_nodes))+list(extra_seeds))
        else:
            shuffled_nodes = np.random.permutation(range(n_nodes))
        list_of_seeds = [shuffled_nodes[i:i + r] for i in range(0, len(shuffled_nodes), r)]
        for seeds in list_of_seeds:
            x = np.zeros((n_nodes,r))
            y_actual = np.zeros((n_nodes,r))
            y_Bourgain = np.zeros((n_nodes,r))
            y_Sarma1 = np.zeros((n_nodes,r))
            y_Sarma2 = np.zeros((n_nodes,r))
            y_Sarma3 = np.zeros((n_nodes,r))
            for i in range(r):
                u = seeds[i]
                x[u,i] = 1
                y_actual[:,i] = y_actual_all[:,u]
                y_Bourgain[:,i] = y_Bourgain_all[:,u]
                y_Sarma1[:,i] = y_Sarma1_all[:,u]
                y_Sarma2[:,i] = y_Sarma2_all[:,u]
                y_Sarma3[:,i] = y_Sarma3_all[:,u]
            samples_x.append(torch.tensor(x.astype(np.float32), requires_grad=True))
            samples_y_actual.append(torch.tensor(y_actual).to(torch.float32))
            samples_y_Bourgain.append(y_Bourgain)
            samples_y_Sarma1.append(y_Sarma1)
            samples_y_Sarma2.append(y_Sarma2)
            samples_y_Sarma3.append(y_Sarma3)
            samples_edge_index.append(edge_index)
        if samples[3] == None:
            samples_weights = None
        else:
            samples_weights = [samples[3][0] for i in range(len(list_of_seeds))]
        y_pred = predict(gpu_bool, model, criterion_type, samples_x, samples_edge_index, samples_weights)
        title = 'GNN, out_channels = floor(sqrt(n))'
   
    y_pred = np.array(list(chain(*[np.ravel(arr) for arr in y_pred])))
    y_actual = np.array(list(chain(*[np.ravel(arr) for arr in samples_y_actual])))
    if n <= 200:
        y_Bourgain = np.array(list(chain(*[np.ravel(arr) for arr in samples_y_Bourgain])))
        y_Sarma1 = np.array(list(chain(*[np.ravel(arr) for arr in samples_y_Sarma1])))
        y_Sarma2 = np.array(list(chain(*[np.ravel(arr) for arr in samples_y_Sarma2])))
        y_Sarma3 = np.array(list(chain(*[np.ravel(arr) for arr in samples_y_Sarma3])))

    mseGNN = mean_squared_error(y_actual, y_pred)
    mseBourgain = mean_squared_error(y_actual, y_Bourgain)
    mseSarma1 = mean_squared_error(y_actual, y_Sarma1)
    mseSarma2 = mean_squared_error(y_actual, y_Sarma2)
    mseSarma3 = mean_squared_error(y_actual, y_Sarma3)
    
    if display_results:

        diff_pred = y_actual - y_pred
        values, base = np.histogram(diff_pred, bins=100)
        cumulative = np.cumsum(values)
        plt.plot(base[:-1], cumulative, label=title, alpha = 0.75)

        if n <= 200:

            diff_Bourgain = y_actual-y_Bourgain
            values, base = np.histogram(diff_Bourgain, bins=100)
            cumulative = np.cumsum(values)
            plt.plot(base[:-1], cumulative, label='Bourgain', alpha = 0.75)

            diff_Sarma1 = y_actual-y_Sarma1
            values, base = np.histogram(diff_Sarma1, bins=100)
            cumulative = np.cumsum(values)
            plt.plot(base[:-1], cumulative, label='Sarma, k = 1', alpha = 0.75)

            diff_Sarma2 = y_actual-y_Sarma2
            values, base = np.histogram(diff_Sarma2, bins=100)
            cumulative = np.cumsum(values)
            plt.plot(base[:-1], cumulative, label='Sarma, k = 2', alpha = 0.75)

            diff_Sarma3 = y_actual-y_Sarma3
            values, base = np.histogram(diff_Sarma3, bins=100)
            cumulative = np.cumsum(values)
            plt.plot(base[:-1], cumulative, label='Sarma, k = 3', alpha = 0.75)
            
        plt.xlabel("Actual Distance - Predicted Distance")
        plt.ylabel("Cummulative Frequency")
        plt.title("All Distances: n = "+str(n)+", lambda = "+str(lbd))
        plt.legend()
        filenames = os.listdir(dir)
        if '0.png' not in filenames:
            plt.savefig(dir+'/0.png')
        else:
            filenames = sorted([int(f[:-4]) for f in filenames])
            plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
        plt.show()

        plt.scatter(y_actual, y_pred, label=title, alpha = 0.1)
        plt.xlabel("Actual Distance")
        plt.ylabel('Predicted Distance')
        plt.title("All Distances: n = "+str(n)+", lambda = "+str(lbd))
        filenames = os.listdir(dir)
        if '0.png' not in filenames:
            plt.savefig(dir+'/0.png')
        else:
            filenames = sorted([int(f[:-4]) for f in filenames])
            plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
        plt.show()

        plt.hist(y_actual, edgecolor='white', alpha=0.4)
        #sns.kdeplot(y_actual, fill=True, alpha=0.2)
        plt.xlabel("Actual Distance")
        plt.ylabel('Density')
        plt.title("All Distances: n = "+str(n)+", lambda = "+str(lbd))
        filenames = os.listdir(dir)
        if '0.png' not in filenames:
            plt.savefig(dir+'/0.png')
        else:
            filenames = sorted([int(f[:-4]) for f in filenames])
            plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
        plt.show()

        #plt.hist(diff_pred, edgecolor='white', alpha=0.4)
        sns.kdeplot(diff_pred, fill=True, alpha=0.2)
        plt.xlabel("Actual Distance - Predicted Distance")
        plt.ylabel('Density')
        plt.title("All Distances: n = "+str(n)+", lambda = "+str(lbd))
        filenames = os.listdir(dir)
        if '0.png' not in filenames:
            plt.savefig(dir+'/0.png')
        else:
            filenames = sorted([int(f[:-4]) for f in filenames])
            plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')   
        plt.show()

        print(title+', MSE:',mseGNN)
        print('Bourgain, MSE:',mseBourgain)
        print('Sarma, k = 1, MSE:',mseSarma1)
        print('Sarma, k = 2, MSE:',mseSarma2)
        print('Sarma, k = 3, MSE:',mseSarma3)

    return [mseGNN,mseBourgain,mseSarma1,mseSarma2,mseSarma3]

In [ ]:
dir = 'outputs/lbd 1 new dump copy'
lbd = 1

In [ ]:
mse1 = []
mse2 = []
graph_sizes = 2**np.array(range(12))*10
for n in graph_sizes:
    print(n)
    samples = generateERSamples(200,50,50,n,lbd)
    model1 = run(samples,'gcn','mse','adam','cyclic-cosine',100,20)
    model2 = run_out1(samples,'gcn','mse','adam','cyclic-cosine',100,20)
    evaluate(n,model1,model2,'mse',samples)
    list1 = []
    list2 = []
    one_sample = generate_one_ER_all_distances(n,lbd,True)
    list1.append(evaluate_one_graph_all_distances(n,model1,'mse',one_sample,True))
    list2.append(evaluate_one_graph_all_distances(n,model2,'mse',one_sample,True))
    for k in range(100):
        one_sample = generate_one_ER_all_distances(n,lbd)
        list1.append(evaluate_one_graph_all_distances(n,model1,'mse',one_sample))
        list2.append(evaluate_one_graph_all_distances(n,model2,'mse',one_sample))
    mse1.append(np.mean(np.array(list1), axis=0))
    mse2.append(np.mean(np.array(list2), axis=0))

In [ ]:
mse1 = np.array(mse1)
plt.plot(range(12),mse1[:,0], label = 'GNN, out_channels = floor(sqrt(n))')
plt.plot(range(12),mse1[:,1], label = 'Bourgain')
plt.plot(range(12),mse1[:,2], label = 'Sarma, k = 1')
plt.plot(range(12),mse1[:,3], label = 'Sarma, k = 2')
plt.plot(range(12),mse1[:,4], label = 'Sarma, k = 3')
custom_ticks = range(12)
custom_labels = graph_sizes
plt.xticks(custom_ticks, custom_labels,rotation=45)
plt.xlabel("Graph Size")
plt.ylabel('MSE')
plt.title("Mean All-Distance MSE: lambda = "+str(lbd))
plt.legend()
filenames = os.listdir(dir)
if '0.png' not in filenames:
    plt.savefig(dir+'/0.png')
else:
    filenames = sorted([int(f[:-4]) for f in filenames])
    plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')
plt.show()

mse2 = np.array(mse2)
plt.plot(range(12),mse1[:,0], label = 'GNN, out_channels = 1')
plt.plot(range(12),mse1[:,1], label = 'Bourgain')
plt.plot(range(12),mse1[:,2], label = 'Sarma, k = 1')
plt.plot(range(12),mse1[:,3], label = 'Sarma, k = 2')
plt.plot(range(12),mse1[:,4], label = 'Sarma, k = 3')
custom_ticks = range(12)
custom_labels = graph_sizes
plt.xticks(custom_ticks, custom_labels,rotation=45)
plt.xlabel("Graph Size")
plt.ylabel('MSE')
plt.title("Mean All-Distance MSE: lambda = "+str(lbd))
plt.legend()
filenames = os.listdir(dir)
if '0.png' not in filenames:
    plt.savefig(dir+'/0.png')
else:
    filenames = sorted([int(f[:-4]) for f in filenames])
    plt.savefig(dir+'/'+str(filenames[-1]+1)+'.png')
plt.show()

In [ ]:
def allShortestDistances_directed(i,j,d_ki,d_ij,d_jk): ## assume i,j in all shortest paths
    n = len(d_ki)
    distances =  np.zeros((n,n))
    for u in range(n):
        for v in range(n):
            if u != v:
                distances[u,v] = d_ki[u]+d_ij+d_jk[v]
    distances[i,j] = min(distances[i,j],d_ij)
    return distances

def allShortestDistances_undirected(i,j,d_ki,d_jk):
    if i == j:
        d_ij = 0
    else:
        d_ij = min(d_ki[j],d_jk[i])
    n = len(d_ki)
    distances =  np.zeros((n,n))
    for u in range(n):
        for v in range(u+1,n):
            distances[u,v] = min(d_ki[u]+d_ki[v],d_jk[u]+d_jk[v],d_ki[u]+d_ij+d_jk[v],d_ki[v]+d_ij+d_jk[u])
            distances[v,u] = distances[u,v]
    distances[i,j] = min(distances[i,j],d_ij)
    distances[j,i] = distances[i,j]
    return distances

def ShortestPath(G,distances,u,v,epsilon):
    S = [u,v]
    for k in range(distances.shape[0]):
        if np.abs(distances[u,k]+distances[k,v]-distances[u,v]) <= epsilon:
            S.append(k)
    S.sort()
    if not isinstance(G, np.ndarray):
        subgraph = G.subgraph(S)
        if nx.has_path(subgraph, u, v):
            path = nx.shortest_path(subgraph, u, v, weight="weight")
            distance = nx.shortest_path_length(subgraph, u, v, weight="weight")
            return path, distance
        else:
            return None, None
    else:
        subgraph = G[S]
        subgraph = subgraph[:,S]
        subgraph = matrix_to_graph(subgraph)
        u_masked = S.index(u)
        v_masked = S.index(v)
        if nx.has_path(subgraph, u_masked, v_masked):
            path = nx.shortest_path(subgraph, u_masked, v_masked, weight="weight")
            distance = nx.shortest_path_length(subgraph, u_masked, v_masked, weight="weight")
            return [S[i] for i in path], distance
        else:
            return None, None

In [ ]:
## Sarma claimed that there is at least one common node in SKETCH[u] and SKETCH[v] for undirected connected graphs. This is FALSE!!
import numpy as np
def checkSarmaAssumption(n,k):
    r = int(np.floor(np.log(n))) ## which base? use e instead of 10 this time since it returns a higher r, lowering the possibility of having no common nodes.
    seed_sizes = 2**np.array(list(range(r+1))*k)
    max_nodes_to_considered = (np.sum(seed_sizes)+1)*2
    boolean = n>max_nodes_to_considered
    return boolean
k1 = []
k2 = []
k3 = []
for n in range(1,100000):
    k1.append(checkSarmaAssumption(n,1))
    k2.append(checkSarmaAssumption(n,2))
    k3.append(checkSarmaAssumption(n,3))
print(np.sum(k1))
print(np.sum(k2))
print(np.sum(k3))

In [ ]:
def offlineSample_GNN(model,criterion_type,G,u,weighted = False):

    n_nodes = len(G.nodes())
    if n_nodes <= 1:
        return None, set()
    r = np.floor(np.log(n_nodes))
    sample_sets = [np.random.choice(G.nodes(),size=2**i,replace=False) for i in range(r+1)]

    gpu_bool = torch.cuda.is_available()
    out = model.out_channels
    x = np.zeros((n_nodes,out))
    x[u,0] = 1
    if out > 1:
        extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
        for i in range(out-1):
            x[extra_seeds[i],i+1] = 1
    samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
    samples_edge_index = [torch.tensor(np.array(list(G.edges())).T).to(torch.int64)]
    if weighted:
        samples_weights = [torch.tensor(list(nx.get_edge_attributes(G,'weight').values())).to(torch.float32)]
    else:
        samples_weights = None
    y_pred = predict(gpu_bool, model, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
    distances = [[y_pred[i] for i in S] for S in sample_sets]
    index = [d.index(min(d)) for d in distances]
    closest_points = [(sample_sets[i][index[i]],distances[i][index[i]]) for i in range(r+1)]
    return closest_points,set(np.concatenate(sample_sets))

def offlineSketch_GNN(model,criterion_type,G,u,k,weighted=False):
    closest_points,sample_sets = offlineSample_GNN(model,criterion_type,G,u,weighted)
    for i in range(k):
        closest_points_new,sample_sets_new = offlineSample_GNN(model,criterion_type,G,u,weighted)
        closest_points = closest_points.union(closest_points_new)
        sample_sets = sample_sets.union(sample_sets_new)
    return np.array(list(closest_points)),np.array(list(sample_sets))

def onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,k,weighted=False): ## upper bound
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    sketch_u,_ = offlineSketch_GNN(model1,criterion_type,G,u,k,weighted)
    sketch_v,_ = offlineSketch_GNN(model2,criterion_type,G,v,k,weighted)
    if sketch_u.shape[0] != 0 and sketch_v.shape[0] != 0:
        common_nodes = [w for w in sketch_u[:,0] if w in sketch_v[:,0]]
        while None in common_nodes:
            common_nodes.remove(None)
        min_dist = float('inf')
        for w in common_nodes:
            dist = sketch_u[sketch_u[:, 0] == w][0,1] + sketch_v[sketch_v[:, 0] == w][0,1]
            if dist < min_dist:
                min_dist = dist
        return min_dist
    else:
        return float('inf')

def onlineShortestPath_Bourgain_GNN(model1,model2,criterion_type,G,u,v,weighted=False): ## lower bound

    n_nodes = len(G.nodes())
    if n_nodes <= 1:
        return None, set()
    r = np.floor(np.log(n_nodes))
    sample_sets = [np.random.choice(G.nodes(),size=2**i,replace=False) for i in range(r+1)]

    gpu_bool = torch.cuda.is_available()
    samples_edge_index = [torch.tensor(np.array(list(G.edges())).T).to(torch.int64)]
    if weighted:
        samples_weights = [torch.tensor(list(nx.get_edge_attributes(G,'weight').values())).to(torch.float32)]
    else:
        samples_weights = None
    if model1 == model2: # set model1 = model2 if undirected
        out = model1.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            x[v,1] = 1
            extra_seeds = np.random.choice(G.nodes(),size=out-2,replace=True)
            for i in range(out-2):
                x[extra_seeds[i],i+2] = 1
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0]
            y_pred_u = y_pred[:,0]
            y_pred_v = y_pred[:,1]
        if out == 1:
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred_u = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
            x = np.zeros((n_nodes,out))
            x[v,0] = 1
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred_v = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        d_u_S = [[y_pred_u[i] for i in S] for S in sample_sets]
        d_v_S = [[y_pred_v[i] for i in S] for S in sample_sets]
        to_remove = [idx for idx,val in enumerate(list(zip(d_u_S,d_v_S))) if val[0] == float('inf') or val[1] == float('inf')]
        d_u_S = np.array([value for index, value in enumerate(d_u_S) if index not in to_remove])
        d_v_S = np.array([value for index, value in enumerate(d_v_S) if index not in to_remove])
        return np.max(np.abs(d_u_S-d_v_S))
    else:
        out = model1.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_u_k = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        x = np.zeros((n_nodes,out))
        x[v,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_v_k = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        out = model2.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_k_u = predict(gpu_bool, model2, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        x = np.zeros((n_nodes,out))
        x[v,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_k_v = predict(gpu_bool, model2, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        d_u_S = [[y_pred_u_k[i] for i in S] for S in sample_sets]
        d_v_S = [[y_pred_v_k[i] for i in S] for S in sample_sets]
        d_S_u = [[y_pred_k_u[i] for i in S] for S in sample_sets]
        d_S_v = [[y_pred_k_v[i] for i in S] for S in sample_sets]
        to_remove = [idx for idx,val in enumerate(list(zip(d_u_S,d_v_S))) if val[0] == float('inf') or val[1] == float('inf')]
        d_u_S = np.array([value for index, value in enumerate(d_u_S) if index not in to_remove])
        d_v_S = np.array([value for index, value in enumerate(d_v_S) if index not in to_remove])
        to_remove = [idx for idx,val in enumerate(list(zip(d_S_u,d_S_v))) if val[0] == float('inf') or val[1] == float('inf')]
        d_S_u = np.array([value for index, value in enumerate(d_S_u) if index not in to_remove])
        d_S_v = np.array([value for index, value in enumerate(d_S_v) if index not in to_remove])
        return max([0,np.max(d_S_v-d_S_u),np.max(d_u_S-d_v_S)])

def shortestDistance_allNodes_Sarma_GNN(model1,model2,criterion_type,G,u,k,weighted=False):
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    distances = np.zeros(G.shape[0])
    for v in range(G.shape[0]):
        if u != v:
            distances[v] = onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,k,weighted)
    return distances

def shortestDistance_allNodes_Bourgain_GNN(model1,model2,criterion_type,G,u,weighted=False):
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    distances = np.zeros(G.shape[0])
    for v in range(G.shape[0]):
        if u != v:
            distances[v] = onlineShortestPath_Bourgain_GNN(model1,model2,criterion_type,G,u,v,weighted)
    return distances

def shortestDistance_allNodes_networkx(G,u):
    if isinstance(G, np.ndarray):
        G = matrix_to_graph(G)
    n_nodes = len(G.nodes())
    distances = np.zeros(n_nodes)
    for v in range(n_nodes):
        if u != v:
            if nx.has_path(G, u, v):
                distances[v] = nx.shortest_path_length(G, u, v, weight="weight")
            else:
                distances[v] = float('inf')       
    return distances